In [1]:
pip install PyPDF2 pdfplumber nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.4 MB/s eta 0:00:00


In [22]:
import os
import re
import PyPDF2
import pdfplumber
from pathlib import Path
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Descargar recursos de NLTK si no están disponibles
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/pupunkt_tabnkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('punkt_tab')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

class PDFProcessor:
    def __init__(self, input_folder, output_folder):
        """
        Inicializa el procesador de PDFs

        Args:
            input_folder: Carpeta con los PDFs a procesar
            output_folder: Carpeta donde se guardarán los archivos procesados
        """
        self.input_folder = Path(input_folder)
        self.output_folder = Path(output_folder)
        self.output_folder.mkdir(exist_ok=True)

        # Configurar stopwords en español
        self.stop_words = set(stopwords.words('spanish'))

    def extract_text_from_pdf(self, pdf_path):
        """
        Extrae texto de un archivo PDF usando pdfplumber como método principal
        y PyPDF2 como respaldo
        """
        text = ""

        try:
            # Intentar con pdfplumber primero (mejor para PDFs complejos)
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
        except:
            # Si falla, intentar con PyPDF2
            try:
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        text += page.extract_text() + "\n"
            except Exception as e:
                print(f"Error al leer {pdf_path}: {str(e)}")
                return ""

        return text

    def clean_text(self, text):
        """
        Limpia el texto según los criterios especificados
        """
        # Convertir a minúsculas
        text = text.lower()

        # Eliminar números de página (patrones comunes)
        # Elimina números solos en una línea o números con formato "Página X"
        text = re.sub(r'^\d+\s*$', '', text, flags=re.MULTILINE)
        text = re.sub(r'página\s*\d+', '', text, flags=re.IGNORECASE)
        text = re.sub(r'pp' '\s*\d+', '', text, flags=re.IGNORECASE)
        text = re.sub(r'p\.\s*\d+', '', text, flags=re.IGNORECASE)
        text = re.sub(r'pág\.\s*\d+', '', text, flags=re.IGNORECASE)
        text = re.sub(r'^-\s*\d+\s*-\s*$', '', text, flags=re.MULTILINE)

        # También manejar casos donde hay espacios antes del salto de línea
        # Como "nac- ieron" en la misma línea
        text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)

        # Eliminar patrones tipo peerj.preprints.27580v1 o similares
        # Este patrón captura: palabra.palabra.número+letra+número
        text = re.sub(r'\b\w+\.\w+\.\d+[a-zA-Z]+\d*\b', ' ', text)

        # Eliminar patrones de identificadores de documentos científicos
        # Como: arxiv.1234.5678, doi.10.1234/5678, pmid.12345678
        text = re.sub(r'\b\w+\.\d+\.\d+\b', ' ', text)
        text = re.sub(r'\bdoi\.\borg\.\S+', ' ', text)
        text = re.sub(r'\bpmid\.\d+', ' ', text)
        text = re.sub(r'\barxiv\.\d+\.\d+', ' ', text)


        # Eliminar números con comas como 45, 56, 78
        # Este patrón elimina secuencias de números separados por comas
        text = re.sub(r'\b\d+(?:\s*,\s*\d+)+\b', '', text)

        # Eliminar números solos (incluyendo decimales)
        # Esto elimina números aislados como 123, 45.67, etc.
        text = re.sub(r'\b\d+\.?\d*\b', '', text)

        # Eliminar referencias tipo [1], [2,3], [45-47], etc.
        text = re.sub(r'\[\d+(?:[-,]\d+)*\]', '', text)

        # Eliminar años solos (4 dígitos)
        text = re.sub(r'\b\d{4}\b', '', text)

        # casos especiales de limpieza del texto de UNESCO
        #Eliminar números romanos en minúscula
        #roman_numerals_pattern = r'\b(?:i[vx]|v?i{0,3}|x)\b'
        text = re.sub(r'\b(?:i[vx]|v?i{0,3}|x)\b', ' ', text)
        # elimina numeracion del tipo 45ª
        text = re.sub(r'\b\d+ª\b', ' ', text)
        #elimina url que solo terminan con '.com'
        text = re.sub(r'\b(?!https:\/\/|www\.)[a-zA-Z0-9.-]+\.com\b', ' ', text)


        # casos especiales de limpieza del texto de Conocimiento abierto
        # elimina letras solas
        text = re.sub(r'\b[a-z]\b', ' ', text)
        # elimina el siguiente patrón: ccoolleecccciióónn ggrruuppooss ddee ttrraabbaajjoo ccoonnoocciimmiieennttoo aabbiieerrttoo eenn aamméérriiccaa llaattiinnaa
        text = re.sub(r'([a-záéíóúñ])\1+',r'\1', text) # ver si conviene o no, tiene palabras en ingles como access(queda aces) o common(queda comon)
        # filtra las palabras de 2 digitos: Ej:fs, ar, eb
        text = re.sub(r'\b[a-z]{2}\b', ' ', text)
        # filtra las palabras de 3 digitos sin vocales: Ej:pcb
        text = re.sub(r'\b[bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ]{3}\b', ' ', text)
        # filtra las asociaciones de este tipo: n2o0s2, a0i
        text = re.sub(r'\b(?=\w*\d)(?=\d*\w)\w+\b', ' ', text)



        # Eliminar URLs
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        #text = re.sub(r'[a-zA-Z0-9$-_@.&+!*\\(\\),%]+(?:\.[a-zA-Z]{2,})+', '', text)

        # Remove HTML tags
        text = re.sub ( r'<.*?>' ,'', text)

        # Eliminar correos electrónicos
        text = re.sub(r'\S+@\S+', '', text)

        # Eliminar caracteres especiales excesivos pero mantener puntuación básica
        # Mantener: . , ; : ! ? ¿ ¡ ( ) - " '
        # text = re.sub(r'[^\w\s\.\,\;\:\!\?\¿\¡\(\)\-\"\ª\'áéíóúñÁÉÍÓÚÑ]', '', text)
        # decido eliminar todos los caracteres especiales por eso comento lo anterior
        text = re.sub(r'[^\w\s]', '', text)

        # Eliminar múltiples espacios
        text = re.sub(r'\s+', ' ', text)

        # Eliminar múltiples saltos de línea
        text = re.sub(r'\n+', '\n', text)

        # Eliminar líneas que solo contienen espacios
        lines = text.split('\n')
        lines = [line.strip() for line in lines if line.strip()]
        text = '\n'.join(lines)

        return text

    # def tokenize_text(self, text, remove_stopwords=True, remove_punctuation=True):
    #     """
    #     Tokeniza el texto y opcionalmente elimina stopwords y puntuación

    #     Args:
    #         text: Texto a tokenizar
    #         remove_stopwords: Si eliminar palabras vacías
    #         remove_punctuation: Si eliminar puntuación
    #     """
    #     # Tokenizar
    #     tokens = word_tokenize(text, language='spanish')

    #     # Filtrar tokens
    #     filtered_tokens = []
    #     for token in tokens:
    #         # Eliminar puntuación si se especifica
    #         if remove_punctuation and token in string.punctuation:
    #             continue

    #         # Eliminar stopwords si se especifica
    #         if remove_stopwords and token.lower() in self.stop_words:
    #             continue

    #         # Eliminar tokens de un solo carácter (excepto 'a' y 'o' que pueden ser significativos)
    #         if len(token) == 1 and token.lower() not in ['a', 'o']:
    #             continue

    #         # Eliminar tokens que son solo números
    #         if token.isdigit():
    #             continue

    #         filtered_tokens.append(token)

    #     return filtered_tokens


    def tokenize_text(self, text, remove_punctuation=True):
      """
      Tokeniza el texto y siempre elimina stopwords del español y números/dígitos solos.
      Opcionalmente, elimina la puntuación.

      Args:
          text (str): Texto a tokenizar.
          remove_punctuation (bool): Si eliminar puntuación (True por defecto).

      Returns:
          list: Lista de tokens limpios.
      """

      # Tokenizar el texto
      tokens = word_tokenize(text, language='spanish')

      ## Convertir a minúsculas para una comparación consistente
      # ya lo hago en la sección de limpieza
      #tokens = [token.lower() for token in tokens]

      filtered_tokens = []
      for token in tokens:
          # Eliminar stopwords de español (siempre)
          if token in self.stop_words:
              continue

          # Eliminar números/dígitos solos (siempre)
          if token.isdigit():
              continue

          # Eliminar tokens de un solo carácter (excepto 'a' y 'o' que pueden ser significativos)
          # Ya que eliminamos puntuación antes, esto se enfoca más en letras sueltas
          if len(token) == 1 and token not in ['a', 'o']:
              continue

          # Asegurarse de no añadir cadenas vacías resultantes de la limpieza previa
          if token:
              filtered_tokens.append(token)

      return filtered_tokens





    def save_processed_text(self, text, tokens, filename):
        """
        Guarda el texto procesado y los tokens en archivos separados
        """
        # Crear subcarpetas
        text_folder = self.output_folder / 'cleaned_text'
        tokens_folder = self.output_folder / 'tokens'
        text_folder.mkdir(exist_ok=True)
        tokens_folder.mkdir(exist_ok=True)

        # Guardar texto limpio
        text_path = text_folder / f"{filename}_cleaned.txt"
        with open(text_path, 'w', encoding='utf-8') as f:
            f.write(text)

        # Guardar tokens
        tokens_path = tokens_folder / f"{filename}_tokens.txt"
        with open(tokens_path, 'w', encoding='utf-8') as f:
            f.write(' '.join(tokens))

        # Guardar tokens uno por línea (útil para análisis)
        tokens_lines_path = tokens_folder / f"{filename}_tokens_lines.txt"
        with open(tokens_lines_path, 'w', encoding='utf-8') as f:
            for token in tokens:
                f.write(token + '\n')

        return text_path, tokens_path

    def process_single_pdf(self, pdf_path):
        """
        Procesa un único archivo PDF
        """
        print(f"\nProcesando: {pdf_path.name}")

        # Extraer texto
        print("  - Extrayendo texto...")
        raw_text = self.extract_text_from_pdf(pdf_path)

        if not raw_text:
            print(f"  - ERROR: No se pudo extraer texto de {pdf_path.name}")
            return None

        # Limpiar texto
        print("  - Limpiando texto...")
        cleaned_text = self.clean_text(raw_text)

        # Tokenizar
        print("  - Tokenizando...")
        tokens = self.tokenize_text(cleaned_text, remove_punctuation=True)
        #tokens = self.tokenize_text(cleaned_text, remove_stopwords=True, remove_punctuation=True)

        # Guardar resultados
        print("  - Guardando resultados...")
        filename = pdf_path.stem  # Nombre sin extensión
        text_path, tokens_path = self.save_processed_text(cleaned_text, tokens, filename)

        # Mostrar estadísticas
        print(f"  - Estadísticas:")
        print(f"    * Caracteres en texto original: {len(raw_text)}")
        print(f"    * Caracteres en texto limpio: {len(cleaned_text)}")
        print(f"    * Número de tokens: {len(tokens)}")
        print(f"    * Tokens únicos: {len(set(tokens))}")

        return {
            'filename': filename,
            'original_chars': len(raw_text),
            'cleaned_chars': len(cleaned_text),
            'total_tokens': len(tokens),
            'unique_tokens': len(set(tokens)),
            'text_path': text_path,
            'tokens_path': tokens_path
        }

    def process_all_pdfs(self):
        """
        Procesa todos los PDFs en la carpeta de entrada
        """
        # Buscar todos los archivos PDF
        pdf_files = list(self.input_folder.glob('*.pdf'))

        if not pdf_files:
            print(f"No se encontraron archivos PDF en {self.input_folder}")
            return []

        print(f"Se encontraron {len(pdf_files)} archivos PDF para procesar")

        results = []
        for pdf_path in pdf_files:
            result = self.process_single_pdf(pdf_path)
            if result:
                results.append(result)

        # Generar resumen
        self.generate_summary(results)

        return results

    def generate_summary(self, results):
        """
        Genera un resumen del procesamiento
        """
        if not results:
            return

        summary_path = self.output_folder / 'processing_summary.txt'

        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write("RESUMEN DE PROCESAMIENTO DE PDFs\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Total de archivos procesados: {len(results)}\n\n")

            total_original = sum(r['original_chars'] for r in results)
            total_cleaned = sum(r['cleaned_chars'] for r in results)
            total_tokens = sum(r['total_tokens'] for r in results)

            f.write(f"Estadísticas globales:\n")
            f.write(f"  - Caracteres totales (original): {total_original:,}\n")
            f.write(f"  - Caracteres totales (limpio): {total_cleaned:,}\n")
            f.write(f"  - Reducción de caracteres: {(1 - total_cleaned/total_original)*100:.1f}%\n")
            f.write(f"  - Tokens totales: {total_tokens:,}\n\n")

            f.write("Detalle por archivo:\n")
            f.write("-" * 50 + "\n")

            for r in results:
                f.write(f"\n{r['filename']}.pdf\n")
                f.write(f"  - Caracteres originales: {r['original_chars']:,}\n")
                f.write(f"  - Caracteres limpios: {r['cleaned_chars']:,}\n")
                f.write(f"  - Tokens totales: {r['total_tokens']:,}\n")
                f.write(f"  - Tokens únicos: {r['unique_tokens']:,}\n")

        print(f"\nResumen guardado en: {summary_path}")

def calculate_word_frequencies(tokens_file, top_n=50):
    """
    Calcula las frecuencias de palabras desde un archivo de tokens
    """
    from collections import Counter

    with open(tokens_file, 'r', encoding='utf-8') as f:
        tokens = f.read().split()

    # Calcular frecuencias
    word_freq = Counter(tokens)

    # Obtener las palabras más comunes
    most_common = word_freq.most_n(top_n)

    return word_freq, most_common

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
#import nltk
#nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
# Ejemplo de uso
if __name__ == "__main__":
    # Configurar rutas
    input_folder = "pdfs"  # Carpeta con los PDFs
    output_folder = "processed_pdfs"  # Carpeta de salida

    # Crear procesador
    processor = PDFProcessor(input_folder, output_folder)

    # Procesar todos los PDFs
    results = processor.process_all_pdfs()

    # Ejemplo: Calcular frecuencias de palabras del primer archivo procesado
    if results:
        first_result = results[0]
        tokens_file = first_result['tokens_path']

        print(f"\n\nFrecuencias de palabras para {first_result['filename']}:")
        print("-" * 50)

        word_freq, most_common = calculate_word_frequencies(tokens_file, top_n=20)

        for word, count in most_common:
            print(f"{word}: {count}")

Se encontraron 2 archivos PDF para procesar

Procesando: Recomendación de la UNESCO sobre la Ciencia Abierta.pdf
  - Extrayendo texto...
  - Limpiando texto...
  - Tokenizando...
  - Guardando resultados...
  - Estadísticas:
    * Caracteres en texto original: 77715
    * Caracteres en texto limpio: 65808
    * Número de tokens: 5981
    * Tokens únicos: 1724

Procesando: Conocimiento_abierto_en_america_latina_trayectorias_y_desafios.pdf
  - Extrayendo texto...
  - Limpiando texto...
  - Tokenizando...
  - Guardando resultados...
  - Estadísticas:
    * Caracteres en texto original: 622410
    * Caracteres en texto limpio: 499897
    * Número de tokens: 49546
    * Tokens únicos: 9299

Resumen guardado en: processed_pdfs/processing_summary.txt


Frecuencias de palabras para Recomendación de la UNESCO sobre la Ciencia Abierta:
--------------------------------------------------


AttributeError: 'Counter' object has no attribute 'most_n'